In [ ]:
%matplotlib inline


PyTorch sequential classification model example
==================

An example of a sequential network that classifies digit images used as an OpenML flow.
We use sub networks here in order to show the that network hierarchies can be achieved with ease.


In [ ]:
import torch.nn
import torch.optim

import openml
import openml.extensions.pytorch
import openml.extensions.pytorch.layers

import logging

Enable logging in order to observe the progress while running the example.



In [ ]:
openml.config.logger.setLevel(logging.DEBUG)
openml.extensions.pytorch.config.logger.setLevel(logging.DEBUG)

Define a sequential network that does the initial image reshaping
and normalization model.



In [ ]:
processing_net = torch.nn.Sequential(
    openml.extensions.pytorch.layers.Functional(function=torch.Tensor.reshape,
                                                shape=(-1, 1, 28, 28)),
    torch.nn.BatchNorm2d(num_features=1)
)

Define a sequential network that does the extracts the features from the
image.



In [ ]:
features_net = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5),
    torch.nn.LeakyReLU(),
    torch.nn.MaxPool2d(kernel_size=2),
    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
    torch.nn.LeakyReLU(),
    torch.nn.MaxPool2d(kernel_size=2),
)

Define a sequential network that flattens the features and compiles the
results into probabilities for each digit.



In [ ]:
results_net = torch.nn.Sequential(
    openml.extensions.pytorch.layers.Functional(function=torch.Tensor.reshape,
                                                shape=(-1, 4 * 4 * 64)),
    torch.nn.Linear(in_features=4 * 4 * 64, out_features=256),
    torch.nn.LeakyReLU(),
    torch.nn.Dropout(),
    torch.nn.Linear(in_features=256, out_features=10),
)

The main network, composed of the above specified networks.



In [ ]:
model = torch.nn.Sequential(
    processing_net,
    features_net,
    results_net
)

Download the OpenML task for the mnist 784 dataset.



In [ ]:
task = openml.tasks.get_task(3573)

Run the model on the task (requires an API key).



In [ ]:
run = openml.runs.run_model_on_task(model, task, avoid_duplicate_runs=False)
# Publish the experiment on OpenML (optional, requires an API key).
run.publish()

print('URL for run: %s/run/%d' % (openml.config.server, run.run_id))